# Worksheet - Classification (Part II)

## Learning Goals:

After completing this workshop session, you will be able to:

* Describe what a test data set is and how it is used in classification.
* Understand several ways of representing classifier performance: accuracy, precision, and recall, and the confusion matrix.
* Using Python, evaluate classifier performance using a test data set and appropriate metrics.
* Using Python, execute cross-validation in Python to choose the number of neighbours.
* Identify when it is necessary to scale variables before classification and do this using Python
* In a dataset with > 2 attributes, perform k-nearest neighbour classification in Python using the `scikit-learn` package to predict the class of a test dataset.
* Describe advantages and disadvantages of the k-nearest neighbour classification algorithm.

This worksheet covers parts of [Chapter 6](https://python.datasciencebook.ca/classification2) of the online textbook. You should read this chapter before attempting this assignment. Any place you see `___`, you must fill in the function, variable, or data to complete the code. Substitute the `raise NotImplementedError` with your completed code and answers then proceed to run the cell!

In [3]:
### Run this cell before continuing.
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

## 1. Fruit Data Example

In the agricultural industry, cleaning, sorting, grading, and packaging food products are all necessary tasks in the post-harvest process. Products are classified based on appearance, size and shape, attributes which helps determine the quality of the food. Sorting can be done by humans, but it is tedious and time consuming. Automatic sorting could help save time and money. Images of the food products are captured and analysed to determine visual characteristics. 

The [dataset](https://www.kaggle.com/mjamilmoughal/k-nearest-neighbor-classifier-to-predict-fruits/notebook) contains observations of fruit described with four features: (1) mass (in g), (2) width (in cm), (3) height (in cm), and (4) color score (on a scale from 0 - 1).

To get started building a classifier that can classfiy a fruit based on its appearance, use `pd.read_csv` to load the file `fruit_data.csv` (found in the data folder) from the previous tutorial into your notebook.

*Assign your data to an object called `fruit_data`.*

In [6]:
# Run this cell to read the data
fruit_data = pd.read_csv("data/fruit_data.csv")

Let's take a look at the first few observations in the fruit dataset. Run the cell below.

In [7]:
# Run this cell to preview the data
fruit_data.head()

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


Now let's investigate the class counts for each kind of fruit:

In [10]:
fruit_data['fruit_name'].value_counts(normalize=False)

fruit_name
apple       19
orange      19
lemon       16
mandarin     5
Name: count, dtype: int64

We can change to using `normalize=True` to get the class proportions:

In [11]:
fruit_data['fruit_name'].value_counts(normalize=True)

fruit_name
apple       0.322034
orange      0.322034
lemon       0.271186
mandarin    0.084746
Name: proportion, dtype: float64

## Randomness and Setting Seeds

This worksheet uses functions from the `scikit-learn` library, which not only allows us to perform K-nearest neighbour classification, but also allows us to evaluate how well our classification worked. In order to ensure that the steps in the worksheet are reproducible, we need to set a *`random_state`* or *random seed*, i.e., a numerical "starting value," which determines the sequence of random numbers Python will generate.

Below in many cells we have included an argument to set the `random_state` or `np.random.seed`. They are necessary to make sure the autotesting code functions properly. In your own analysis however, it is a better practice to set the *`random_state`* or *random seed* just once at the beginning of your script.

## 2. Splitting the data into a training and test set

In this exercise, we will be partitioning `fruit_data` into a training (75%) and testing (25%) set using the `scikit-learn` package. After creating the test set, we will put the test set away in a lock box and not touch it again until we have found the best k-nn classifier we can make using the training set. We will use the variable `fruit_name` as our class label. 


### Question 1

To create the training and test set, we would use the `train_test_split` function from `scikit-learn` package. Save the trained dataset and test dataset as `fruit_train` and `fruit_test`, respectively. 

In [ ]:
# Randomly take 75% of the data in the training set.
# This will be proportional to the different number of fruit names in the dataset.

# ___, ___ = train_test_split(___, test_size=___, random_state=123) # set the random state to be 123

# your code here
raise NotImplementedError
fruit_train

In [ ]:
fruit_test

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_train is None)).encode("utf-8")+b"ef8b1").hexdigest() == "9f77123ff7006f15c739b7b78f5f2d4de80459f5", "type of fruit_train is None is not bool. fruit_train is None should be a bool"
assert sha1(str(fruit_train is None).encode("utf-8")+b"ef8b1").hexdigest() == "7ed9c577a01bef83e68375a03929f6c1cba7755b", "boolean value of fruit_train is None is not correct"

assert sha1(str(type(fruit_test is None)).encode("utf-8")+b"ef8b2").hexdigest() == "b0edefb6186b34edf34d83f7bcf5cbb59e46ea56", "type of fruit_test is None is not bool. fruit_test is None should be a bool"
assert sha1(str(fruit_test is None).encode("utf-8")+b"ef8b2").hexdigest() == "96404ff84888a103c5f891759de786517fe317e2", "boolean value of fruit_test is None is not correct"

assert sha1(str(type(fruit_train.shape)).encode("utf-8")+b"ef8b3").hexdigest() == "ad58db68d7f455473cab66cff03bdba319b16f11", "type of fruit_train.shape is not tuple. fruit_train.shape should be a tuple"
assert sha1(str(len(fruit_train.shape)).encode("utf-8")+b"ef8b3").hexdigest() == "1d7f9a2b05a377135a71d8c34ecb0a5926601bb4", "length of fruit_train.shape is not correct"
assert sha1(str(sorted(map(str, fruit_train.shape))).encode("utf-8")+b"ef8b3").hexdigest() == "9e1e7bdacecb3a487b64e0f64d1f61366a6ea4fe", "values of fruit_train.shape are not correct"
assert sha1(str(fruit_train.shape).encode("utf-8")+b"ef8b3").hexdigest() == "342376346f8cf46351b13bed9b0160a140dd9fbb", "order of elements of fruit_train.shape is not correct"

assert sha1(str(type(fruit_test.shape)).encode("utf-8")+b"ef8b4").hexdigest() == "c00a1e9b6f64d1629e4f464c1935eda7b0f96279", "type of fruit_test.shape is not tuple. fruit_test.shape should be a tuple"
assert sha1(str(len(fruit_test.shape)).encode("utf-8")+b"ef8b4").hexdigest() == "6c6e5d05f1db9c0aef7b920ba2cfd609e1a2d2d1", "length of fruit_test.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test.shape))).encode("utf-8")+b"ef8b4").hexdigest() == "ade72264f11f0c795f1ca06d6df199248d2fda40", "values of fruit_test.shape are not correct"
assert sha1(str(fruit_test.shape).encode("utf-8")+b"ef8b4").hexdigest() == "191ee842577393f5745b2c256304dcdd2895b4a6", "order of elements of fruit_test.shape is not correct"

assert sha1(str(type(sum(fruit_train.mass))).encode("utf-8")+b"ef8b5").hexdigest() == "d43052ac9a4c7109e3fd472091a2a67541dc4590", "type of sum(fruit_train.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_train.mass)).encode("utf-8")+b"ef8b5").hexdigest() == "03b793f8288b890907d114552b291e4cb42a95f0", "value of sum(fruit_train.mass) is not correct"

assert sha1(str(type(sum(fruit_test.mass))).encode("utf-8")+b"ef8b6").hexdigest() == "21310037fbeb70908577535bbc83ff4f6b2a975a", "type of sum(fruit_test.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_test.mass)).encode("utf-8")+b"ef8b6").hexdigest() == "baabdedf68419227ffd00e67f81a343e9c77d3e5", "value of sum(fruit_test.mass) is not correct"

print('Success!')

### Question 2 

K-nearest neighbors is sensitive to the scale of the predictors so we should do some preprocessing to standardize them. Remember that standardizing involves centering/shifting (subtracting the mean of each variable) and scaling (dividing by its standard deviation). Also remember that standardization is *part of your training procedure*, so you can't use your test data to compute the centered / scaled values for each variable. Therefore, you must pass only the training data to your preprocessor to compute the preprocessing steps. This ensures that our test data does not influence any aspect of our model training. Once we have created the standardization preprocessor, we can then later on apply it separately to both the training and test data sets.

For this exercise, let's see if `mass` and `color_score` can predict `fruit_name`. 

To scale and center the data, first, pass the predictors to the `make_column_transformer` function to make the preprocessor.

*Assign your answer to an object called `fruit_preprocessor`.*

In [ ]:
# ___ = make_column_transformer(
#     (___, [___, ___]),
#     verbose_feature_names_out=False
# )

# your code here
raise NotImplementedError
fruit_preprocessor

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_preprocessor is None)).encode("utf-8")+b"36d09").hexdigest() == "c16043f1e011ae5e24d73f767d244e153de052d1", "type of fruit_preprocessor is None is not bool. fruit_preprocessor is None should be a bool"
assert sha1(str(fruit_preprocessor is None).encode("utf-8")+b"36d09").hexdigest() == "a36c675b8869eb47d14d616bcfc17bf0679c1480", "boolean value of fruit_preprocessor is None is not correct"

assert sha1(str(type(type(fruit_preprocessor))).encode("utf-8")+b"36d0a").hexdigest() == "5b4a6271e3d19f04ad8e3a02389bdde94d645e8f", "type of type(fruit_preprocessor) is not correct"
assert sha1(str(type(fruit_preprocessor)).encode("utf-8")+b"36d0a").hexdigest() == "33fc23392075e2159cb275f5b072c2bd3eef06f2", "value of type(fruit_preprocessor) is not correct"

assert sha1(str(type(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"36d0b").hexdigest() == "c2ee407b203b6e1c1faf4f5bbefb51099c64d2fa", "type of fruit_preprocessor.transformers[0][0] is not str. fruit_preprocessor.transformers[0][0] should be an str"
assert sha1(str(len(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"36d0b").hexdigest() == "b66e860a0a8f53af64caf60afdbf123a56eddbfa", "length of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0].lower()).encode("utf-8")+b"36d0b").hexdigest() == "b0dd09d0d913f060dfae366a65bdc73cc4c6dbe9", "value of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0]).encode("utf-8")+b"36d0b").hexdigest() == "b0dd09d0d913f060dfae366a65bdc73cc4c6dbe9", "correct string value of fruit_preprocessor.transformers[0][0] but incorrect case of letters"

assert sha1(str(type(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"36d0c").hexdigest() == "462915beb3245eb3f238c0e6100eba8c60d5eeeb", "type of fruit_preprocessor.transformers[0][2] is not list. fruit_preprocessor.transformers[0][2] should be a list"
assert sha1(str(len(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"36d0c").hexdigest() == "dbcd1005f772edc467986417157dca8731db4c76", "length of fruit_preprocessor.transformers[0][2] is not correct"
assert sha1(str(sorted(map(str, fruit_preprocessor.transformers[0][2]))).encode("utf-8")+b"36d0c").hexdigest() == "280af5a8b2775780fabf4d84c5f5db2cfdbdbe56", "values of fruit_preprocessor.transformers[0][2] are not correct"
assert sha1(str(fruit_preprocessor.transformers[0][2]).encode("utf-8")+b"36d0c").hexdigest() == "2b1472a2c7fea5769561b8e3eef648c0052e5e8a", "order of elements of fruit_preprocessor.transformers[0][2] is not correct"

print('Success!')

Now that we have split the data, we can do things like exploratory data analysis and model fitting. Before we move onto the latter, run the cell below to visualize the two of the predictors (mass in grams, and width in cm) as a scatter plot, colouring the observations by their class labels.

In [ ]:
# Create the scatterplot
fruit_chart = alt.Chart(fruit_data).mark_point(size=15).encode(
    x=alt.X("mass").title("Mass (grams)"),
    y=alt.Y("width")
        .title("Width (cm)")
        .scale(zero=False),
    color=alt.Color("fruit_name").title("Fruit")
)

fruit_chart

## 3. Cross-validation

### Question 3

The vast majority of predictive models in statistics and machine learning have parameters that you have to pick. For the past few exercises, we have had to pick the number of neighbours for the class vote, which we have done arbitraily. But, is it possible to make this selection, *i.e., tune the model, in a principled way?* Ideally, we want to pick the number of neighborurs to maximize the performance of our classifier on data *it hasn’t seen yet*.

An important aspect of the tuning process is that we can, if we want to, split our training data again, train and evaluate a classifier for each split, and then choose the parameter based on all of the different results. If we just split our training data once, our best parameter choice will depend strongly on the randomness from how this single split was made. Using multiple different splits, we’ll get a more robust estimate of accuracy, which will lead to a more suitable choice of the number of neighbours $K$ to perform well on unseen data.

The idea of training and evaluating models on multiple training data splits times is called "cross-validation". In cross-validation, we split our overall training data into $C$ evenly-sized chunks, and then iteratively use 1 chunk as the **validation set** and combine the remaining $C−1$ chunks as the **training set.** The validation set is used in a similar was as the test set, **except** that the test set is only used once at the end to report model performance whether we use model performance on the validation set to select the model during cross-validation. 

---

We can perform a cross-validation in Python using the `cross_validate` function from the `scikit-learn` package. To use this function, you have to identify the model, the training set, and specify the `cv` parameter (the number of folds $C$, defaults to 5). We should set `return_train_score` to be `True` to return the training score as well.

Before we use the `cross_validate` function, we need to perform the pipeline analysis again. You can reuse the `X` and `y` variables you constructed from the training data earlier, as well as the `fruit_preprocessor`and `knn_spec` variables. However, you will need to create a new pipeline since the one we made earlier is already fitted on all the data and here we want to fit it on different splits of the data during cross-validation. Since the cross_validate function outputs a dictionary, we use `pd.DataFrame` to convert it to a dataframe for convenience, as in the textbook.

*Assign your answer to an object called `fruit_vfold_score`*.

In [ ]:
np.random.seed(2020)  # DO NOT REMOVE

# ___ = ___(fruit_preprocessor, knn_spec)
# ___ = pd.___(
#     cross_validate(
#         estimator=___,
#         cv=5,
#         X=___,
#         y=___,
#         return_train_score=True,
#     )
# )

# your code here
raise NotImplementedError
fruit_vfold_score

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_vfold_score is None)).encode("utf-8")+b"36a22").hexdigest() == "b1fc58d5e23c9423bbf8765714e5e58804fce66a", "type of fruit_vfold_score is None is not bool. fruit_vfold_score is None should be a bool"
assert sha1(str(fruit_vfold_score is None).encode("utf-8")+b"36a22").hexdigest() == "d2cc03f8ff44ef13491fa232070b6de92b811475", "boolean value of fruit_vfold_score is None is not correct"

assert sha1(str(type(fruit_vfold_score)).encode("utf-8")+b"36a23").hexdigest() == "d2dcb89307585b23fa4ca6f8a76f8c2c0f23346f", "type of type(fruit_vfold_score) is not correct"

assert sha1(str(type(fruit_vfold_score.shape)).encode("utf-8")+b"36a24").hexdigest() == "546d04c6fccac7326ab7400bb48a82c394c90130", "type of fruit_vfold_score.shape is not tuple. fruit_vfold_score.shape should be a tuple"
assert sha1(str(len(fruit_vfold_score.shape)).encode("utf-8")+b"36a24").hexdigest() == "d8cd8be65e2d915f492f7185bcd2736d2f68a197", "length of fruit_vfold_score.shape is not correct"
assert sha1(str(sorted(map(str, fruit_vfold_score.shape))).encode("utf-8")+b"36a24").hexdigest() == "653cb8e4264891bd349087f0a6ae4d134fd597c4", "values of fruit_vfold_score.shape are not correct"
assert sha1(str(fruit_vfold_score.shape).encode("utf-8")+b"36a24").hexdigest() == "89f531606aaf9ca14b74ef68296d4ab26838067f", "order of elements of fruit_vfold_score.shape is not correct"

assert sha1(str(type(fruit_pipe is None)).encode("utf-8")+b"36a25").hexdigest() == "9541e0d3962315e7cfa81d63c9ff1179e06dbe63", "type of fruit_pipe is None is not bool. fruit_pipe is None should be a bool"
assert sha1(str(fruit_pipe is None).encode("utf-8")+b"36a25").hexdigest() == "020dfe9202b64ad0e6861ad87373f929d736eefc", "boolean value of fruit_pipe is None is not correct"

assert sha1(str(type(type(fruit_pipe))).encode("utf-8")+b"36a26").hexdigest() == "fd97847bfa2e68ff8df95e6074f5a234c145fd58", "type of type(fruit_pipe) is not correct"
assert sha1(str(type(fruit_pipe)).encode("utf-8")+b"36a26").hexdigest() == "656864c949d89ae0be260be526c38fa4039065bd", "value of type(fruit_pipe) is not correct"

assert sha1(str(type(len(fruit_pipe.named_steps))).encode("utf-8")+b"36a27").hexdigest() == "bbcdb9c08b17d3d8a66fe4a53686f010ae0bccbb", "type of len(fruit_pipe.named_steps) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(fruit_pipe.named_steps)).encode("utf-8")+b"36a27").hexdigest() == "33ba12d9299d1f48e8d73ede50813d18a77ff874", "value of len(fruit_pipe.named_steps) is not correct"

assert sha1(str(type(fruit_pipe.named_steps.keys())).encode("utf-8")+b"36a28").hexdigest() == "ea5bb9eb37991a1ed3ecb7735822ec397000ce42", "type of fruit_pipe.named_steps.keys() is not correct"
assert sha1(str(fruit_pipe.named_steps.keys()).encode("utf-8")+b"36a28").hexdigest() == "577fdf89e2c84a15d65fa7cec8269b13b6314a4e", "value of fruit_pipe.named_steps.keys() is not correct"

print('Success!')

### Question 4

Now that we have ran a cross-validation on each train/validation split, one has to ask, how accurate was the classifier's validation across the folds? We can aggregate the *mean* and *standard error* of these scores from each folds. The standard error is essentially a measure of how uncertain we are in the mean value. Use the `agg` dataframe method to compute both the mean and the standard error; make sure the first row of the dataframe contains the mean values and the second contains the standard error values.

*Assign your answer to an object called `fruit_metrics`.*

In [ ]:
# ___ = fruit_vfold_score.___([___, ___])


# your code here
raise NotImplementedError
fruit_metrics

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_metrics.shape)).encode("utf-8")+b"7608a").hexdigest() == "8e5f57d0f7ed0aa98c690da117537d080d1bff1a", "type of fruit_metrics.shape is not tuple. fruit_metrics.shape should be a tuple"
assert sha1(str(len(fruit_metrics.shape)).encode("utf-8")+b"7608a").hexdigest() == "f68c65a025bd3924f48305894fadea89e0c5b46e", "length of fruit_metrics.shape is not correct"
assert sha1(str(sorted(map(str, fruit_metrics.shape))).encode("utf-8")+b"7608a").hexdigest() == "bbca15d44552fa5e34cd23e46733915b6043b927", "values of fruit_metrics.shape are not correct"
assert sha1(str(fruit_metrics.shape).encode("utf-8")+b"7608a").hexdigest() == "f8ddb51af4648fc9c62d3c1d3a78fc8589ac74ca", "order of elements of fruit_metrics.shape is not correct"

assert sha1(str(type(fruit_metrics.test_score)).encode("utf-8")+b"7608b").hexdigest() == "4fae85143b5b445ce9b9807dfe9a1a379dc90033", "type of fruit_metrics.test_score is not correct"
assert sha1(str(fruit_metrics.test_score).encode("utf-8")+b"7608b").hexdigest() == "6924635698218860ac3c086773cb0acfa29dace4", "value of fruit_metrics.test_score is not correct"

print('Success!')

## 5. Parameter value selection

Using a 5-fold cross-validation, we have established a prediction accuracy for our classifier. If we were to improve our classifier, we would like to try different number of neighbours, $K$. Then we could use cross-validation to calculate an accuracy for each value of $K$ in a reasonable range, and pick the value of $K$ that gives us the best accuracy on the validation data. 

The great thing about the `scikit-learn` package is that it provides functions to conveniently tune parameters such as $K$ by training and evaluating models (via crossvalidation) for a range of specified values of $K$. The function we will use here is called "exhaustive grid search" ([sklearn.model_selection.GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

### Question 5

Create a new K-nearest neighbor model specification but instead of specifying a particular value for the `n_neighbors` argument, try exploring a range of values with `GridSearchCV`. Before we use `GridSearchCV`, we should define the grid of values that we want to explore, and redefine the pipeline without specifying a particular value of $K$. To save us some time, instruct the grid search to use 4-fold cross-validation, rather than the default 5-fold.

*Assign your answer to an object called `knn_tune_grid`.* 

In [ ]:
### Run this cell
param_grid = {
    "kneighborsclassifier__n_neighbors": range(2, 15, 1),
}
fruit_tune_pipe = make_pipeline(fruit_preprocessor, KNeighborsClassifier())

In [ ]:
# ___ = GridSearchCV(
#     ___, ___, ___=__,
# )


# your code here
raise NotImplementedError
knn_tune_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_tune_grid is None)).encode("utf-8")+b"de7e3").hexdigest() == "b8f0b73e45acd1ecde4311b54cb8ca5c4fd48116", "type of knn_tune_grid is None is not bool. knn_tune_grid is None should be a bool"
assert sha1(str(knn_tune_grid is None).encode("utf-8")+b"de7e3").hexdigest() == "22ff6a02d8a0748a0942a49f18a56f89c44071a4", "boolean value of knn_tune_grid is None is not correct"

assert sha1(str(type(type(knn_tune_grid))).encode("utf-8")+b"de7e4").hexdigest() == "bba351be309dda4ac68b4ebd1aafb92b79462f94", "type of type(knn_tune_grid) is not correct"
assert sha1(str(type(knn_tune_grid)).encode("utf-8")+b"de7e4").hexdigest() == "5daba4ef43b4a410d3c641b8d3b2e56daed064e6", "value of type(knn_tune_grid) is not correct"

assert sha1(str(type(knn_tune_grid.param_grid.keys())).encode("utf-8")+b"de7e5").hexdigest() == "ff297bec527b81362f07bc67cd01011f1add1a08", "type of knn_tune_grid.param_grid.keys() is not correct"
assert sha1(str(knn_tune_grid.param_grid.keys()).encode("utf-8")+b"de7e5").hexdigest() == "1c7370822141bce722ca65597af73993dba4a560", "value of knn_tune_grid.param_grid.keys() is not correct"

assert sha1(str(type(knn_tune_grid.estimator.named_steps.keys())).encode("utf-8")+b"de7e6").hexdigest() == "dd0e8c205b661479a0048e81ead835f12b2b5ddf", "type of knn_tune_grid.estimator.named_steps.keys() is not correct"
assert sha1(str(knn_tune_grid.estimator.named_steps.keys()).encode("utf-8")+b"de7e6").hexdigest() == "9fe3afc262d1fef9058a7a3342aa8dbc9a76685c", "value of knn_tune_grid.estimator.named_steps.keys() is not correct"

assert sha1(str(type(knn_tune_grid.cv)).encode("utf-8")+b"de7e7").hexdigest() == "947f9f92f52034f3258a2e920f111db2ce0fc28b", "type of knn_tune_grid.cv is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_tune_grid.cv).encode("utf-8")+b"de7e7").hexdigest() == "048c41e7d49846e06e4228f89a17df11f2732e82", "value of knn_tune_grid.cv is not correct"

print('Success!')

### Question 6

Now, let's fit the grid search object to the data, using the `X` and `y` variables we created earlier.

*Assign your tuned model to a variable called `knn_model_grid`.*

Next, from `knn_model_grid`, find out the `cv_results_` and save it in a dataframe. 

*Assign your answer to a variable called `accuracies_grid`.*

In [ ]:
# ___ = ___.fit(___, ___)

# ___ = pd.DataFrame(___.cv_results_)

# your code here
raise NotImplementedError
accuracies_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(type(knn_model_grid))).encode("utf-8")+b"5aaa1").hexdigest() == "7800ffeeffbe3f109596728a651492f72ed697b9", "type of type(knn_model_grid) is not correct"
assert sha1(str(type(knn_model_grid)).encode("utf-8")+b"5aaa1").hexdigest() == "a777b6bc7a96e92dc37d7e48e532c4403b013440", "value of type(knn_model_grid) is not correct"

assert sha1(str(type(accuracies_grid is None)).encode("utf-8")+b"5aaa2").hexdigest() == "2bbc8cb94c17fac722104bbe0ae8178d6283ea7d", "type of accuracies_grid is None is not bool. accuracies_grid is None should be a bool"
assert sha1(str(accuracies_grid is None).encode("utf-8")+b"5aaa2").hexdigest() == "b31adcd31362204c2f193a8d747bac72afd4a47a", "boolean value of accuracies_grid is None is not correct"

assert sha1(str(type(accuracies_grid)).encode("utf-8")+b"5aaa3").hexdigest() == "a67afa6224ce1811c7d07895e39f81d73e309929", "type of type(accuracies_grid) is not correct"

assert sha1(str(type(accuracies_grid.shape)).encode("utf-8")+b"5aaa4").hexdigest() == "07ba20ae9b6524ef5c009590c807287bad2b6ca8", "type of accuracies_grid.shape is not tuple. accuracies_grid.shape should be a tuple"
assert sha1(str(len(accuracies_grid.shape)).encode("utf-8")+b"5aaa4").hexdigest() == "e362cae886bd94094b80f36e4d1c28b67e1ca7d7", "length of accuracies_grid.shape is not correct"
assert sha1(str(sorted(map(str, accuracies_grid.shape))).encode("utf-8")+b"5aaa4").hexdigest() == "2a86bd742543c1a3f2d3750c8ae31e564709e00e", "values of accuracies_grid.shape are not correct"
assert sha1(str(accuracies_grid.shape).encode("utf-8")+b"5aaa4").hexdigest() == "bcd625ee3af5d1cd105b2b2335288631494670e5", "order of elements of accuracies_grid.shape is not correct"

assert sha1(str(type(sum(accuracies_grid.mean_test_score))).encode("utf-8")+b"5aaa5").hexdigest() == "f04499ac18fececdea66994f437f6fe195a9a404", "type of sum(accuracies_grid.mean_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.mean_test_score), 2)).encode("utf-8")+b"5aaa5").hexdigest() == "fef13114978598bb2dd1245e857f4ad0cd359ce9", "value of sum(accuracies_grid.mean_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.std_test_score))).encode("utf-8")+b"5aaa6").hexdigest() == "90280827318185a637233f593624365d40176a3a", "type of sum(accuracies_grid.std_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.std_test_score), 2)).encode("utf-8")+b"5aaa6").hexdigest() == "7ea1bce420fa1d5037e56e6b0e1b99bf030cc962", "value of sum(accuracies_grid.std_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors))).encode("utf-8")+b"5aaa7").hexdigest() == "bcd0d8dac389990c3750e6af054592990b173ab3", "type of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors)).encode("utf-8")+b"5aaa7").hexdigest() == "8805dfb33692c14619c01c7dd91f5135cbda75dd", "value of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not correct"

print('Success!')

### Visualizing a parameter's effect on model performance

Visually inspecting the grid search results can help us find the best value for the number of neighbors parameter.

Create a line plot using the `accuracies_grid` dataframe with `param_kneighborsclassifier__n_neighbors` on the x-axis and the `mean_test_score` on the y-axis. Use `point=True` to include a point for each value of $K$. Make it an effective visualization.

*Assign your answer to a variable called `accuracy_versus_k_grid`.*

In [ ]:
# ___ = alt.Chart(___).mark_line(___).encode(
#     x=alt.X(___)
#         .title(___)
#         .scale(zero=False),
#     y=alt.Y(___)
#         .title(___)
#         .scale(zero=False)
# )

## 6. Understanding prediction errors

Now, let's look at the *confusion matrix* for the classifier. This will show us a table comparing the predicted labels with the true labels. 

A confusion matrix is essentially a classification matrix. The columns of the confusion matrix represent the actual class and the rows represent the predicted class (or vice versa). Shown below is an example of a confusion matrix.

|                    | Predicted Positive | Predicted Negative |
|--------------------|:------------------:|:------------------:|
| **Truly Positive** | True Positive      |     False Negative |
| **Truly Negative** | False Positive     |      True Negative |


- A **true positive** is an outcome where the model correctly predicts the positive class.
- A **true negative** is an outcome where the model correctly predicts the negative class.
- A **false positive** is an outcome where the model incorrectly predicts the positive class.
- A **false negative** is an outcome where the model incorrectly predicts the negative class.

<br>

We can create a confusion matrix by using the `crosstab` function from `pandas`. In the dataframe created by `crosstab`, the true labels will be to the left, and the predicted labels will be on top (as in the matrix above). In contrast to the confusion matrix above where there are only two possible outcomes (positive/negative), we have four possible outcomes (the four fruit names). Therefore, our dataframe will be bigger than the matrix above and contain 16 possible outcomes instead of 4.

*Assign your answer to an object called `fruit_mat`*.

In [ ]:
# ___ = pd.___(
#     fruit_test_predictions[___],  # True labels
#     fruit_test_predictions[___],  # Predicted labels
# )

# your code here
raise NotImplementedError
fruit_mat

With many observations, it can be difficult to interpret the confusion matrix when it is presented as a table like above. In these cases, we could instead use the `ConfusionMatrixDisplay` function of the `scikit-learn` package to visualize the confusion matrix as a heatmap. Please run the cell below to see the fruit confusion matrix as a heatmap.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    fruit_fit,  # We are directly passing the pipeline and let sklearn do the predictions for us
    X_test,
    y_test
)

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_mat is None)).encode("utf-8")+b"6d49c").hexdigest() == "a56e492407ffda77dae01cf693d7cec3160d24a5", "type of fruit_mat is None is not bool. fruit_mat is None should be a bool"
assert sha1(str(fruit_mat is None).encode("utf-8")+b"6d49c").hexdigest() == "c7f4085c7df1435511dabd1fe07a5e3f0bf6a04a", "boolean value of fruit_mat is None is not correct"

assert sha1(str(type(fruit_mat)).encode("utf-8")+b"6d49d").hexdigest() == "28c9794a963e59bc5a7cf363c4c0e2789544fc33", "type of type(fruit_mat) is not correct"

assert sha1(str(type(fruit_mat.to_numpy().sum())).encode("utf-8")+b"6d49e").hexdigest() == "01e9653b50d767f4025c03eca241b30e06583876", "type of fruit_mat.to_numpy().sum() is not correct"
assert sha1(str(fruit_mat.to_numpy().sum()).encode("utf-8")+b"6d49e").hexdigest() == "bf73171c60376a7a7ab9275366e3d8a810042a38", "value of fruit_mat.to_numpy().sum() is not correct"

print('Success!')

## Model generalizability

Once you have finished selecting your single, final model (**and only then!**) you may wish to estimate how well your model will generalize to unseen data. You have kept your test set for just that purpose, and that purpose alone. The first step to doing this is to use your model to predict labels for the test set observations:

Next, we can calculate our prediction metrics on the test set, as well a compute and visualize a confusion matrix. We do expect some drop in performance from what we observed in our training set (because we used that set to train the model), but ideally the model performs similarly.